# Week 2 Exercise 2
## Intro to Database Connections, pandas, and Feature Engineering

In [3]:
# Import needed modules
import pandas as pd
import sqlite3 as sql

## SQL --> Dataframe

In [4]:
# Connect to database (need the current working directory for systems that treat the local path as virtual)
import os
con = sql.connect(os.path.join(os.getcwd(), "synthea_and_county_ga.db"))




In [6]:
# Execute a query and store the results in a "dataframe"
df = pd.read_sql_query("""
    select fips, state_abbr, county_name, pop_est_2020, ed_visits
    from us_counties_ahrf_2021
    where state_abbr = 'GA'

""", con)

In [7]:
# Quick settings
pd.options.display.max_columns = None
df = df.rename(columns=str.lower)
df = df.set_index('fips')

## What's in the dataframe?

In [8]:
df.shape

(159, 4)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 13001 to 13321
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   state_abbr    159 non-null    object
 1   county_name   159 non-null    object
 2   pop_est_2020  159 non-null    int64 
 3   ed_visits     159 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 6.2+ KB


In [10]:
# Display the dataframe
df

,state_abbr,county_name,pop_est_2020,ed_visits
fips,,,,
13001,GA,Appling,18325,7740
13003,GA,Atkinson,8393,0
13005,GA,Bacon,11036,3759
13007,GA,Baker,2971,0
13009,GA,Baldwin,45099,30686
...,...,...,...,...
13313,GA,Whitfield,103837,50008
13315,GA,Wilcox,8502,0
13317,GA,Wilkes,9694,2888


## Create two new features

In [11]:
# First example of feature engineering: Ratio
df['ed_visits_per_capita'] = df['ed_visits'] / df['pop_est_2020']

In [12]:
# Round to 2 decimal places
df['ed_visits_per_capita'] = df['ed_visits_per_capita'].round(2)

In [13]:
# Which counties are 1 standard deviation above the mean?
df['ed_visits_high'] = False
df.loc[df['ed_visits_per_capita'] > df['ed_visits_per_capita'].mean() + df['ed_visits_per_capita'].std(), 'ed_visits_high'] = True

In [14]:
df

,state_abbr,county_name,pop_est_2020,ed_visits,ed_visits_per_capita,ed_visits_high
fips,,,,,,
13001,GA,Appling,18325,7740,0.42,False
13003,GA,Atkinson,8393,0,0.00,False
13005,GA,Bacon,11036,3759,0.34,False
13007,GA,Baker,2971,0,0.00,False
13009,GA,Baldwin,45099,30686,0.68,False
...,...,...,...,...,...,...
13313,GA,Whitfield,103837,50008,0.48,False
13315,GA,Wilcox,8502,0,0.00,False
13317,GA,Wilkes,9694,2888,0.30,False


## Output to CSV (for use in Tableau)

In [12]:
# Output to a CSV file (for use in Tableau)
df.to_csv('w2e2_ed_visits_georgia_counties.csv')